In [11]:
import tensorflow as tf
import numpy as np
import dataset as ds
trainToTestRatio = 0.8

dataset = np.load('faceFeatures_9216_normalized_labels_as_coord.npz')
samples = dataset['data'].shape[0]
sampleSize = dataset['data'].shape[1]
coordsCount = dataset['y'].shape[1]
splitIdx = int(samples*trainToTestRatio)
trainData = dataset['data'][:splitIdx]
trainY = dataset['y'][:splitIdx]
testData = dataset['data'][splitIdx:]
testY = dataset['y'][splitIdx:]

resultDataset = ds.TestsetDB()


In [48]:
outputSize = coordsCount
x=tf.placeholder(tf.float32,shape=[None,sampleSize])
y_=tf.placeholder(tf.float32,shape=[None,outputSize])
x_96_96=tf.reshape(x,[-1,96,96,1])
w_conv1=tf.Variable(tf.truncated_normal([3,3,1,32],stddev=0.1))
b_conv1=tf.Variable(tf.constant(0.1,shape=[32]))
h_conv1=tf.nn.relu(tf.nn.conv2d(x_96_96,w_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)
h_max_pool1=tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

w_conv2=tf.Variable(tf.truncated_normal([3,3,32,64],stddev=0.1))
b_conv2=tf.Variable(tf.constant(0.1,shape=[64]))
h_conv2=tf.nn.relu(tf.nn.conv2d(h_max_pool1,w_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
h_max_pool2=tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

w_conv3=tf.Variable(tf.truncated_normal([2,2,64,128],stddev=0.1))
b_conv3=tf.Variable(tf.constant(0.1,shape=[128]))
h_conv3=tf.nn.relu(tf.nn.conv2d(h_max_pool2,w_conv3,strides=[1,1,1,1],padding='SAME')+b_conv3)
h_max_pool3=tf.nn.max_pool(h_conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

h_pool_flat=tf.reshape(h_max_pool3,[-1,12*12*128])

w_fc1=tf.Variable(tf.truncated_normal([12*12*128,500],stddev=0.1))
b_fc1=tf.Variable(tf.constant(0.1,shape=[500]))
h_fc1=tf.nn.relu(tf.matmul(h_pool_flat,w_fc1)+b_fc1)

w_fc2=tf.Variable(tf.truncated_normal([500,500],stddev=0.1))
b_fc2=tf.Variable(tf.constant(0.1,shape=[500]))
h_fc2=tf.nn.relu(tf.matmul(h_fc1,w_fc2)+b_fc2)

w_fc3=tf.Variable(tf.truncated_normal([500,30],stddev=0.1))
b_fc3=tf.Variable(tf.constant(0.1,shape=[30]))
y=tf.nn.relu(tf.matmul(h_fc2,w_fc3)+b_fc3)

In [49]:
#lossFunc = tf.nn.l2_loss(y - y_)
lossFunc = tf.sqrt(tf.reduce_mean((y - y_)**2))

train_step = tf.train.AdamOptimizer(1e-4).minimize(lossFunc)

In [ ]:
epohs = 1001


init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

trainLoss = []
testLoss = []

for i in range(epohs):
    evalY=y.eval(feed_dict={x:trainData},session=sess)
    evalY[~np.isnan(trainY)]=trainY[~np.isnan(trainY)]
    if i % 100 == 0:
        train = lossFunc.eval(feed_dict={x:trainData, y_:evalY}, session=sess)
        evalTestY=y.eval(feed_dict={x:testData},session=sess)
        evalTestY[~np.isnan(testY)]=testY[~np.isnan(testY)]
        test = lossFunc.eval(feed_dict={x:testData, y_:evalTestY}, session=sess)
        
        print('train:', train, 'test:', test)
    train_step.run(feed_dict={x:trainData, y_:evalY}, session=sess)